In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
import torchvision
import numpy as np
import random
import time

In [3]:
import data_reader
SemData = data_reader.read_data_sets('data', padding=0, shuffle=True, noZero=True)

In [4]:
print(len(SemData.train.sentences))
print(len(SemData.test.sentences))
print(len(SemData.weight))

19584
2181
14


In [5]:
class Params():
    def __init__(self):
        self.n_inputs = 300
        self.n_hidden = 150
        self.n_class = 14 # 15 if noZero==False
        self.batch_size = 128
        self.n_train = 3000
        self.n_display = 100
        self.test_size = 2181

params = Params()

In [6]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.n_inputs = params.n_inputs
        self.n_hidden = params.n_hidden
        self.n_class = params.n_class
        self.batch_size = params.batch_size
        self.rnn = nn.RNN(self.n_inputs, self.n_hidden, bidirectional=True)
        self.lstm = nn.LSTM(self.n_inputs, self.n_hidden, bidirectional=True)
        self.fc = nn.Linear(self.n_hidden, self.n_class)
        self.h0 = torch.randn(2, 1, self.n_hidden).cuda()
        self.c0 = torch.randn(2, 1, self.n_hidden).cuda()
    
    def forward(self, sentences):
        output = torch.tensor([]).cuda()
        for i in range(len(sentences)):
            sen_len = sentences[i].shape[0]
            sentence = sentences[i].reshape(1, sen_len, self.n_inputs)
            sentence = torch.tensor(sentence.transpose(1, 0, 2)).cuda()
            # _, h = self.rnn(sentence, self.h0)
            # _, (h, _) = self.lstm(sentence, (self.h0, self.c0))
            # h = torch.nn.functional.dropout(h, 0.5)
            # out = self.fc(h[0])
            
            # h_fb, (_, _) = self.lstm(sentence, (self.h0, self.c0))
            h_fb, _ = self.rnn(sentence, self.h0)
            h, _ = torch.max(h_fb[:, :, 0:self.n_hidden]+h_fb[:, :, self.n_hidden:], 0)
            
            h = torch.nn.functional.dropout(h, 0.3)
            out = self.fc(h)
            output = torch.cat((output, out), 0)
        return output

In [7]:
model = RNN()
model = model.cuda()
criterion = nn.CrossEntropyLoss(weight=torch.tensor(SemData.weight, dtype=torch.float).cuda())
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.01)

In [8]:
def cond_mat(mat, preds, labels):
    for i in range(len(labels)):
        mat[preds[i], labels[i]] += 1
    return mat

In [9]:
def train():
    start = time.clock()
    for step in range(params.n_train):
        train_sentences, train_labels = SemData.train.next_batch(params.batch_size)
        train_labels = torch.tensor(train_labels).cuda()
        optimizer.zero_grad()
        train_output = model(train_sentences)
        _, train_golden = torch.max(train_labels, 1)
        train_loss = criterion(train_output, train_golden)
        train_loss.backward()
        optimizer.step()
        if step%params.n_display == 0 or step == params.n_train-1 :
            print("<step: %d>" % (step))
            train_mat = torch.zeros(14, 14).cuda() # 15 if onZero==False
            _, train_preds = torch.max(train_output, 1)
            train_mat = cond_mat(train_mat, train_preds, train_golden)
            train_accuracy = torch.trace(train_mat) / params.batch_size
            print("train_accuracy: %2.4f %% local_loss: %.8f" % (train_accuracy*100, train_loss.item()))
            
            test_sentences = SemData.test.sentences
            test_labels = torch.tensor(SemData.test.labels).cuda()
            
            test_output = model(test_sentences)
            
            test_mat = torch.zeros(14, 14).cuda() # 15 if onZero==False
            _, test_preds = torch.max(test_output, 1)
            _, test_golden = torch.max(test_labels, 1)
            test_mat = cond_mat(test_mat, test_preds, test_golden)
            test_loss = criterion(test_output, test_golden)
            test_accuracy = torch.trace(test_mat) / params.test_size
            # f1 score (micro)
            TP = torch.tensor([test_mat[i, i] for i in range(14)]).cuda()
            FP = torch.sum(test_mat, 1) - TP
            FN = torch.sum(test_mat, 0) - TP
            P = TP / (TP + FP)
            R = TP / (TP + FN)
            test_f1 = torch.mean(2 / (1/P+1/R))
            print("test_accuracy:  %2.4f %%, total_loss: %.8f, f1_score: %.4f" % (test_accuracy*100, test_loss.item(), test_f1))
    print("------------------------------------")
    print("training time: ", time.clock()-start, " s")

In [10]:
train()

<step: 0>
train_accuracy: 21.0938 % local_loss: 2.88826799
tensor([[403.,   2., 140., 201.,  32.,  46.,  23., 221.,   2.,   7., 128.,  30.,
          15.,   8.],
        [  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [  2.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [  3.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [  2.,   0.,   2.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   3.,   0.,
           0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [ 77.,   1.,  19.,  37.,   7.,   6.,   3.,  57.,   0.,   2.,  32.,   4.,
           0.,   1.],
        [ 68.,   0.,  31.,  36.,   7.,   3.,   1.,  50.,   0.,   1.,  25.,   8.,
           5.,   3.],
        [  0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [ 10.,

<step: 600>
train_accuracy: 50.7812 % local_loss: 1.18220782
tensor([[339.,   1.,  21.,  23.,   3.,  14.,   0.,  62.,   0.,   0.,   9.,   1.,
           1.,   1.],
        [ 25.,   0.,  12.,  12.,   2.,   1.,   1.,   8.,   1.,   0.,   5.,   0.,
           0.,   1.],
        [ 31.,   1.,  69.,  21.,   2.,   1.,   0.,   6.,   1.,   1.,  16.,   0.,
           1.,   1.],
        [ 22.,   0.,  34., 173.,   1.,   3.,   0.,  23.,   0.,   4.,  49.,   1.,
           1.,   3.],
        [ 29.,   0.,   7.,  18.,  37.,   5.,   0.,  17.,   0.,   0.,   3.,   2.,
           0.,   1.],
        [ 72.,   2.,   7.,   3.,   3.,  19.,   0.,  33.,   1.,   0.,   7.,   1.,
           2.,   0.],
        [ 21.,   0.,   2.,   3.,   1.,   0.,  30.,  15.,   0.,   0.,   4.,   0.,
           0.,   0.],
        [ 61.,   0.,  15.,   8.,   0.,   8.,   0., 162.,   0.,   4.,   8.,   6.,
           1.,   1.],
        [ 12.,   0.,   3.,   6.,   0.,   0.,   0.,  10.,   0.,   0.,   4.,   0.,
           1.,   0.],
        [  0

<step: 1200>
train_accuracy: 53.9062 % local_loss: 0.98493379
tensor([[399.,   1.,  16.,  31.,   6.,  11.,   2.,  76.,   1.,   1.,   7.,   2.,
           2.,   1.],
        [  3.,   0.,   0.,   0.,   0.,   0.,   0.,   6.,   1.,   0.,   1.,   0.,
           0.,   0.],
        [ 45.,   1., 124.,  34.,   2.,   2.,   0.,  11.,   0.,   2.,  25.,   0.,
           0.,   1.],
        [ 18.,   0.,  34., 177.,   4.,   0.,   0.,  20.,   0.,   3.,  49.,   1.,
           1.,   1.],
        [ 23.,   0.,   3.,   9.,  43.,   1.,   0.,   6.,   0.,   0.,   5.,   1.,
           0.,   2.],
        [130.,   2.,  11.,   9.,   1.,  31.,   0.,  63.,   1.,   2.,   9.,   5.,
           5.,   1.],
        [  9.,   0.,   1.,   2.,   0.,   0.,  28.,   6.,   0.,   0.,   2.,   0.,
           0.,   0.],
        [ 30.,   0.,   7.,   3.,   0.,   4.,   0., 169.,   0.,   1.,   6.,   1.,
           0.,   0.],
        [  0.,   0.,   0.,   2.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [  

<step: 1800>
train_accuracy: 67.9688 % local_loss: 0.58180541
tensor([[401.,   1.,  11.,  20.,   2.,   7.,   0.,  46.,   0.,   0.,   4.,   0.,
           1.,   0.],
        [  3.,   0.,   0.,   0.,   1.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [ 52.,   2., 132.,  32.,   2.,   3.,   0.,  17.,   0.,   2.,  23.,   0.,
           0.,   2.],
        [ 22.,   0.,  20., 191.,   3.,   1.,   0.,  15.,   0.,   2.,  28.,   3.,
           1.,   2.],
        [ 26.,   0.,   2.,   8.,  45.,   1.,   0.,   8.,   0.,   0.,   4.,   2.,
           0.,   0.],
        [ 70.,   1.,   3.,   8.,   0.,  35.,   1.,  28.,   2.,   1.,   2.,   2.,
           4.,   1.],
        [  5.,   0.,   0.,   0.,   0.,   0.,  30.,   5.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [ 45.,   0.,   7.,   5.,   1.,   1.,   0., 210.,   1.,   1.,   5.,   3.,
           1.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [  